In [24]:
import requests
import hmac
from datetime import datetime
import time
import hashlib
import hmac
import os
import json
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

In [19]:
api_key = os.getenv("BYBIT_API_KEY")
secret_key = os.getenv("BYBIT_SECRET_KEY")

httpClient = requests.Session()
recv_window = str(5000)
url = "https://api-testnet.bybit.com"  # Testnet endpoint


def HTTP_Request(endPoint, method, payload):
    global time_stamp
    time_stamp = str(int(time.time() * 10 ** 3))
    signature = genSignature(payload)
    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    }
    response = httpClient.request(method, url + endPoint + "?" + payload, headers=headers)
    return response.text
    # print(response.headers)


def genSignature(payload):
    param_str = str(time_stamp) + api_key + recv_window + payload
    hash = hmac.new(bytes(secret_key, "utf-8"), param_str.encode("utf-8"), hashlib.sha256)
    signature = hash.hexdigest()
    return signature


start_date = int(datetime.strptime("2024-12-01", "%Y-%m-%d").timestamp())
payload = f"category=linear&symbol=BTCUSD&interval=D&limit=90&start={start_date}"

response = HTTP_Request("/v5/market/kline", "GET", payload)

In [31]:
# > list[0]: startTime	string	Start time of the candle (ms)
# > list[1]: openPrice	string	Open price
# > list[2]: highPrice	string	Highest price
# > list[3]: lowPrice	string	Lowest price
# > list[4]: closePrice	string	Close price. Is the last traded price when the candle is not closed
# > list[5]: volume	string	Trade volume. Unit of contract: pieces of contract. Unit of spot: quantity of coins
# > list[6]: turnover	string	Turnover. Unit of figure: quantity of quota coin
df = pd.DataFrame(json.loads(response)["result"]["list"]).set_axis(["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"], axis=1).sort_values("startTime")
# Convert startTime from unix format to format like "2024-12-01 00:00:00"
df["startTime"] = pd.to_datetime(df["startTime"], unit="ms")


/var/folders/c5/yws7l37x505_stzsyd569pn80000gn/T/ipykernel_82898/1413342274.py:10: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["startTime"] = pd.to_datetime(df["startTime"], unit="ms")


In [32]:
df

,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
89,2018-12-29,3880,3880,3880,3880,111,0.02860824
88,2018-12-30,3880,3880,3880,3880,0,0
87,2018-12-31,3880,3880,3880,3880,0,0
86,2019-01-01,3880,3880,3880,3880,0,0
85,2019-01-02,3880,3914,3760.5,3887.5,4235723,1106.62087526
...,...,...,...,...,...,...,...
4,2019-03-24,3978.5,3978.5,3945.5,3964,999420,252.06844119
3,2019-03-25,3964,3972,3848.5,3902.5,1082211,274.07472471
2,2019-03-26,3902.5,3924.5,3880.5,3918.5,103343047,26440.9739616908
1,2019-03-27,3918.5,4040,3911.5,4025.5,105794956,26568.6550349592
